In [52]:
#include <mlpack/xeus-cling.hpp>
#include <mlpack/core.hpp>
#include <mlpack/methods/linear_regression/linear_regression.hpp>
#include <mlpack/core/data/split_data.hpp>
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"

namespace plt = matplotlibcpp;
using namespace mlpack;


In [53]:
arma::mat dataset;
data::DatasetInfo info;
data::Load("housing.csv", dataset,info);

In [54]:
// Print the first 5 rows of the input data.
std::cout << dataset.submat(0, 0, dataset.n_rows - 1 , 5) << std::endl;

        0   1.0000   2.0000   3.0000   4.0000   4.0000
        0   1.0000   2.0000   3.0000   3.0000   3.0000
        0   1.0000   2.0000   3.0000   3.0000   3.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   2.0000   3.0000   4.0000   5.0000
        0   1.0000   1.0000   1.0000   1.0000   1.0000



In [55]:
#include<mlpack/core/data/one_hot_encoding.hpp>
arma::mat encoded_dataset; 
data::OneHotEncoding(dataset, encoded_dataset, info);

In [56]:
const arma::Col<size_t> indices = {'9'}

In [57]:
// Labels are the last row.
arma::rowvec labels =
    arma::conv_to<arma::rowvec>::from(dataset.row(dataset.n_rows - 2));
dataset.shed_row(dataset.n_rows - 2);

In [58]:
arma::mat trainSet, testSet;
arma::rowvec trainLabels, testLabels;


In [59]:

// Split dataset randomly into training set and test set.
data::Split(dataset, labels, trainSet, testSet, trainLabels, testLabels,
    0.2 /* Percentage of dataset to use for test set. */);

In [60]:
using namespace mlpack::regression;

In [61]:
LinearRegression lr(trainSet, trainLabels, 0.0);

In [62]:
lr.Train(trainSet, trainLabels);

In [63]:
arma::rowvec output; 
lr.Predict(testSet, output);

In [64]:
lr.ComputeError(trainSet, trainLabels);

In [65]:
std::cout<<lr.ComputeError(trainSet, trainLabels);

1.00072e+06

In [66]:
std::cout<<testLabels[1] << std:: endl;
std::cout<<output[1]<<std::endl;

2190
1254.97


In [46]:
std::cout<< testLabels.n_cols;

4128

In [47]:
arma::vec test_labels_col = testLabels.t();
arma::vec output_col = output.t();

In [48]:
testLabels.print()

   9.5900e+02   1.3830e+03   3.1860e+03   2.8630e+03   6.2700e+02   3.6500e+02   2.1770e+03   1.8430e+03   2.9200e+03   7.7700e+02   3.5310e+03   7.5200e+02   3.3650e+03   2.1250e+03   2.8400e+02   1.1460e+03   3.6400e+02   1.6030e+03   2.7820e+03   9.5400e+02   3.7400e+02   4.3900e+02   2.1130e+03   4.7800e+02   2.7660e+03   5.0100e+02   6.2400e+02   2.3120e+03   7.6000e+01   7.4700e+02   1.3340e+03   7.8000e+01   4.8000e+01   2.0670e+03   7.0000e+01   7.3900e+02   1.2150e+03   2.4820e+03   1.2840e+03   5.5100e+02   1.9800e+03   1.9630e+03   7.8000e+01   1.1490e+03   1.0510e+03   1.2350e+03   2.1950e+03   7.4500e+02   3.3000e+01   8.0500e+02   1.9610e+03   6.5500e+02   2.6220e+03   3.0920e+03   2.7300e+02   3.7500e+02   1.3940e+03   6.8100e+02   2.1290e+03   2.1130e+03   1.4150e+03   1.5750e+03   8.6900e+02   5.5700e+02   7.8200e+02   4.3900e+02   1.2220e+03   2.5110e+03   2.5350e+03   3.5100e+03   3.8800e+02   3.2460e+03   1.6720e+03   7.0500e+02   1.9350e+03   1.2040e+03   2.3100e+0